In [2]:
# import nisnap
import pyxnat
import xnat
from pyxnat import Interface
import pandas as pd
import nibabel as nib
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go

In [3]:
my_pyxnat = Interface(server="https://xnat.crmbm.univ-amu.fr",
                    user='hdary')

In [4]:
# if os.environ['XNAT_XSI_TYPE'] != 'xnat:projectData':
#     raise Exception('Must be started from an XNAT project.')

# project = os.environ['XNAT_ITEM_ID']

In [5]:
project = 'TERRA_BRAIN_EPINOV'

In [6]:
def get_resources(my_pyxnat, project):

    resources = []
    experiments = my_pyxnat.array.experiments(project_id=project, columns=['ID', 'project', 'label', 'subject_ID']).data
    # For each experiments fetch all the resources associated with it
    for exp in tqdm(experiments):
        j = my_pyxnat.get('{}/{}'.format(exp['URI'], 'resources')).json()
        exp_res = j['ResultSet']['Result']

        if exp_res:
            resources.extend([[exp['project'], exp['ID'], exp['label'], exp['subject_ID'],
                               r['xnat_abstractresource_id'],
                               r['label']] for r in exp_res])
        else:
            resources.append([exp['project'], exp['ID'],  exp['label'], exp['subject_ID'],
                              'No Data', 'No Data'])
    return resources

In [7]:
def get_subjects(my_pyxnat, project):
    dico = {i.id() :i.label() for i in my_pyxnat.select.project(project).subjects()}
    return dico

In [8]:
dico = get_subjects(my_pyxnat, project)

In [9]:
resources = get_resources(my_pyxnat, project)

  0%|          | 0/34 [00:00<?, ?it/s]

100%|██████████| 34/34 [00:00<00:00, 50.85it/s]


In [10]:
dico_resources = {}
for r in resources:
    if r[5] == 'No Data':
        continue
    if r[5] in dico_resources.keys():
        dico_resources[r[5]].append(dico[r[3]])
    else:
        dico_resources[r[5]] = []
        dico_resources[r[5]].append(dico[r[3]])

In [11]:
# Créer un ensemble contenant tous les sujets
sujets = set()
for valeurs in dico_resources.values():
    sujets.update(valeurs)

# Créer une copie du dictionnaire pour conserver les noms des sujets dans les valeurs
dictionnaire_modifie = {colonne: [sujet in valeurs for sujet in sujets] for colonne, valeurs in dico_resources.items()}

# Convertir le dictionnaire modifié en DataFrame
index = list(sujets)
dataframe = pd.DataFrame(dictionnaire_modifie)

dataframe.insert(0, "Sujets", list(sujets))

# Remplacer les index par les noms des sujets
# dataframe.index = list(sujets)


In [22]:
cell_colors = [['white' if j == 0 else 'green' if val else 'red' for val in dataframe[col]] for j, col in enumerate(dataframe.columns)]

fig = go.Figure(data=[go.Table(
    header=dict(values=list(dataframe.columns)),
    cells=dict(values=[dataframe[col] for col in dataframe.columns],
               fill_color=cell_colors)
)])

# Afficher le tableau Plotly
fig.show()

In [237]:
dico_atlas_path = {'AAL3': 'AAL3/AAL3v1_1mm_to_subj.nii.gz',
            'Neudorfer': 'Neudorfer/MDA_990HCP_atlas_labels_0.5mm.nii.gz_to_subj.nii.gz',
            'AAL3_Neudorfer': 'AAL3_Neudorfer/neudorfer_aal3_to_subj.nii.gz',
            'Hypothalamus': 'Hypothalamus/Hypothalamus2Adele_dilated2_resliced_to_subj.nii.gz',
            'atlas_7TAmi_DGN_SBA_v9': '7TAMI/7TAmi_DGN_SBA_v9_to_subj.nii.gz',
            'atlas_7TAmi_adapted_eMorel_v9': '7TAMI/7TAmi_adapted_eMorel_v9_to_subj.nii.gz',
            'atlas_7TAmi_Amygdala': '7TAMI/7TAmi_Amygdala_to_subj.nii.gz',
            'atlas_7TAmi_CIT168_AMYG': '7TAMI/7TAmi_CIT168_AMYG_to_subj.nii.gz'}

dico_atlas_freesurfer_path = {
            'VEP_atlas': 'subject/mri/origin/aparc+aseg.vep.mgz',
            'Destrieux': 'subject/mri/origin/aparc.a2009s+aseg.mgz',
            'Desikan_Killiany': 'subject/mri/origin/aparc_DKTatlas+aseg.mgz'}

dico_atlas_list = dico_atlas_path.copy()
dico_atlas_list = {cle: [] for cle in dico_atlas_list}
dico_atlas_list['session'] = []
dico_atlas_list['subjects'] = []
for k in dico_atlas_freesurfer_path.keys():
    dico_atlas_list[k] = []

dico_atlas_freesurfer_path = {
            'VEP_atlas': 'subject/mri/origin/aparc+aseg.vep.mgz',
            'Destrieux': 'subject/mri/origin/aparc.a2009s+aseg.mgz',
            'Desikan_Killiany': 'subject/mri/origin/aparc_DKTatlas+aseg.mgz'}

In [238]:
dico_atlas = {}
for sub in my_pyxnat.select.projects(project).subjects():
    for exp in sub.experiments():
        dico_atlas[sub.label() + '*' + exp.label()] = {}
        for res in exp.resources():
            if res.label() == 'FREESURFER':
                dico_atlas[sub.label() + '*' + exp.label()]['aparc'] = res.aparc()
                dico_atlas[sub.label() + '*' + exp.label()]['aseg'] = res.aseg()

                for k in dico_atlas_freesurfer_path.keys():
                    if res.files(k + '.csv').get() != []:
                        # file_object = res.files(k + '.csv')[0].attributes()
                        file_path = res.files(k + '.csv')[0].get()
                        df = pd.read_csv(file_path)
                        dico_atlas[sub.label() + '*' + exp.label()]['extraction_' + k] = df

            elif res.label() == 'Atlas':
                for k,v in dico_atlas_path.items():
                    if res.files(v).get() != []:
                        dico_atlas[sub.label() + '*' + exp.label()][k] = True

            elif res.label() == 'Extraction':
                for k in dico_atlas_path.keys():
                    if res.files(k + '.csv').get() != []:
                        # file_object = res.files(k + '.csv')[0].attributes()
                        file_path = res.files(k + '.csv')[0].get()
                        df = pd.read_csv(file_path)
                        dico_atlas[sub.label() + '*' + exp.label()]['extraction_' + k] = df

In [239]:
dico_atlas['01-59-P-V*202204131119'].keys()

dict_keys(['aparc', 'aseg', 'AAL3', 'AAL3_Neudorfer', 'atlas_7TAmi_DGN_SBA_v9', 'atlas_7TAmi_adapted_eMorel_v9', 'atlas_7TAmi_Amygdala', 'atlas_7TAmi_CIT168_AMYG', 'extraction_AAL3_Neudorfer', 'extraction_atlas_7TAmi_Amygdala', 'extraction_atlas_7TAmi_CIT168_AMYG'])

In [240]:
for sub in dico_atlas.keys():
    dico_atlas_list['subjects'].append(sub.split('*')[0])
    dico_atlas_list['session'].append(sub.split('*')[1])
    for atlas in dico_atlas_list.keys():
        if atlas == 'session' or atlas == 'subjects':
            continue
        if atlas in dico_atlas[sub].keys():
            dico_atlas_list[atlas].append(True)
        else:
            dico_atlas_list[atlas].append(False)

In [242]:
my_dataframe = pd.DataFrame(dico_atlas_list)
new_list = [x for x in list(my_dataframe.columns) if x != 'subjects' and x != 'session']
nouvel_ordre_colonnes = ['subjects', 'session'] + new_list
my_dataframe = my_dataframe[nouvel_ordre_colonnes]

In [243]:
cell_colors = [['white' if j == 0 or j == 1 else 'green' if val else 'red' for val in my_dataframe[col]] for j, col in enumerate(my_dataframe.columns)]

fig = go.Figure(data=[go.Table(
    header=dict(values=list(my_dataframe.columns)),
    cells=dict(values=[my_dataframe[col] for col in my_dataframe.columns],
               fill_color=cell_colors)
               )])

fig.show()

Volume

In [336]:
dico_aseg_list = {}
# dico_aseg_list['session'] = []
# dico_aseg_list['subjects'] = []
dico_aseg_list['values'] = []
dico_aseg_list['label'] = []

for sub in dico_atlas.keys():
    # dico_aseg_list['subjects'].append(sub.split('*')[0])
    # dico_aseg_list['session'].append(sub.split('*')[1])
    if 'aseg' in dico_atlas[sub].keys():
        # print(dico_atlas[sub]['aseg'])
        df = dico_atlas[sub]['aseg']
        # Filtrer les lignes avec measurement égal à "Volume_mm3"
        df_volume = df[df['measurement'] == 'Volume_mm3']
        # Supprimer la colonne "measurement"
        df_volume = df_volume.drop(columns=['measurement'])
        # Transposer le DataFrame
        df_transpose = df_volume.set_index('region').T
        # Réinitialiser les index pour obtenir une mise en forme correcte
        df_transpose.reset_index(drop=True, inplace=True)
        dico_aseg_list['label'].append(['subjects', 'session'] + list(df_transpose.columns))
        dico_aseg_list['values'].append([sub.split('*')[0], sub.split('*')[1]] + list(df_transpose.iloc[0]))
        print()

    else:
        # dico_aseg_list['label'].append(['subjects', 'session'] + 59*[0])
        dico_aseg_list['values'].append([sub.split('*')[0], sub.split('*')[1]] + + 59*[0])

In [337]:
my_dataframe2 = pd.DataFrame(dico_aseg_list['values'])
my_dataframe2.columns = dico_aseg_list['label'][0]

In [359]:
my_dataframe3 = my_dataframe2.T
my_dataframe3.columns = my_dataframe3.iloc[0]

# Supprimez la première ligne car elle est maintenant utilisée comme noms de colonne
my_dataframe3 = my_dataframe3[2:]
# my_dataframe3.columns.values[0] = "ROI"


In [360]:
my_dataframe3

subjects,01_55_L_M,01_45_B-N,01_45_B-N,01_85_G-S,01-59-P-V,01-68-P-N,01-75-R-S,01-63-B-K,01_77_B_B,01_86_P-L,...,01-56-C-D,01_84_GM,01-46_L-J,01-78-M-C,01_52_B-L,01_41_CL,01_50_D-A,01-66-L-Q,01_58_T_A,01_73_A_M
BrainSegVol,0.0,0.0,0.0,0.0,1022580.374364,924666.84967,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1072931.244242,0.0,0.0
BrainSegVolNotVent,0.0,0.0,0.0,0.0,980107.281317,903439.346811,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1042570.916668,0.0,0.0
VentricleChoroidVol,0.0,0.0,0.0,0.0,36647.69615,17805.592883,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25576.548073,0.0,0.0
lhCortexVol,0.0,0.0,0.0,0.0,204580.125985,192301.355968,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,204876.401487,0.0,0.0
rhCortexVol,0.0,0.0,0.0,0.0,213219.579237,191040.035488,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,209801.86332,0.0,0.0
lhCerebralWhiteMatterVol,0.0,0.0,0.0,0.0,196262.102864,186254.064186,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,225695.507993,0.0,0.0
rhCerebralWhiteMatterVol,0.0,0.0,0.0,0.0,206063.094817,186594.74436,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,224281.291607,0.0,0.0
CerebralWhiteMatterVol,0.0,0.0,0.0,0.0,402325.197681,372848.808545,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,449976.799601,0.0,0.0
SubCortGrayVol,0.0,0.0,0.0,0.0,44579.309386,47839.710148,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52740.738104,0.0,0.0
TotalGrayVol,0.0,0.0,0.0,0.0,557499.642814,509031.309091,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,566747.807952,0.0,0.0


In [365]:
import pandas as pd
import plotly.express as px

# Supposons que df soit votre DataFrame

# # Supprimez les lignes avec des valeurs manquantes (NaN)
# df = my_dataframe2.dropna()

# # Sélectionnez uniquement les colonnes qui contiennent les régions et les valeurs associées
regions = ['lhCortexVol', 'rhCortexVol', 'lhCerebralWhiteMatterVol', 'rhCerebralWhiteMatterVol', 'CerebralWhiteMatterVol',
           'TotalGrayVol', 'SupraTentorialVol', 'SupraTentorialVolNotVent', 'SupraTentorialVolNotVentVox', 'MaskVol-to-eTIV',
           'lhSurfaceHoles', 'rhSurfaceHoles', 'SurfaceHoles', 'EstimatedTotalIntraCranialVol', 'Left-Lateral-Ventricle',
           'Left-Inf-Lat-Vent', 'Left-Cerebellum-White-Matter', 'Left-Cerebellum-Cortex', 'Left-Thalamus-Proper',
           'Left-Caudate', 'Left-Putamen', 'Left-Pallidum', '3rd-Ventricle', '4th-Ventricle', 'Brain-Stem', 'Left-Hippocampus',
           'Left-Amygdala', 'CSF', 'Left-Accumbens-area', 'Left-VentralDC', 'Left-vessel', 'Left-choroid-plexus', 'Right-Lateral-Ventricle',
           'Right-Inf-Lat-Vent', 'Right-Cerebellum-White-Matter', 'Right-Cerebellum-Cortex', 'Right-Thalamus-Proper',
           'Right-Caudate', 'Right-Putamen', 'Right-Pallidum', 'Right-Hippocampus', 'Right-Amygdala', 'Right-Accumbens-area',
           'Right-VentralDC', 'Right-vessel', 'Right-choroid-plexus', '5th-Ventricle', 'WM-hypointensities', 'non-WM-hypointensities',
           'Left-non-WM-hypointensities', 'Right-non-WM-hypointensities', 'Optic-Chiasm', 'CC_Posterior', 'CC_Mid_Posterior',
           'CC_Central', 'CC_Mid_Anterior', 'CC_Anterior']

# # Melt le DataFrame pour avoir une seule colonne pour les régions et les valeurs associées
# df_melted = df.melt(id_vars=['subjects', 'session'], value_vars=regions, var_name='Region', value_name='Value')

# Tracer les données avec Plotly

df_melted = my_dataframe3.melt(id_vars='subjects', var_name='Sujet', value_name='Valeur')


# fig = px.scatter(df_melted, x='subjects', y= title="mean N")
# fig.show()

# fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species")

# fig = px.scatter(my_dataframe3, x=regions, y='Value', color='subjects', title='Valeurs des régions pour tous les sujets',
#                  labels={'Value': 'Valeurs', 'Region': 'Régions', 'subjects': 'Sujets'},
#                  category_orders={'Region': regions},
#                  width=1000, height=600)

# fig.update_xaxes(tickangle=45)
# fig.show()


KeyError: "The following id_vars or value_vars are not present in the DataFrame: ['subjects']"

In [339]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(my_dataframe2.columns)),
    cells=dict(values=[my_dataframe[col] for col in my_dataframe.columns])
                   )])

# Spécifier les largeurs des colonnes du tableau Plotly
# column_widths = [100] * len(df.columns)  # Par exemple, définissez toutes les colonnes à la même largeur (100 pixels)
# fig.update_traces(columnwidth=column_widths)  # Mettez à jour les largeurs des colonnes

# # Afficher le tableau Plotly avec une barre de défilement horizontale
fig.update_layout(autosize=True, margin=dict(l=50, r=50, t=50, b=50),
                  xaxis=dict(automargin=True, autorange=True, showgrid=False, showticklabels=True))


fig.show()

In [340]:
import pandas as pd
import plotly.graph_objects as go


fig = go.Figure(data=[go.Table(
    header=dict(values=list(aseg.columns)),
    cells=dict(values=[aseg[col] for col in aseg.columns])
)])

fig.show()

In [244]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from functools import reduce

df = dataframe
# Supposons que vous ayez déjà votre DataFrame df

# Définir des valeurs uniques pour les champs de saisie
sujets_uniques = sorted(df['Sujets'].unique())

# Définir des champs de saisie pour chaque colonne de votre DataFrame
sujets = widgets.Dropdown(options=sujets_uniques, description='Sujets:')

# Créer un tableau Plotly initial
initial_temp_df = df.copy()
initial_fig = go.Figure(data=[go.Table(
    header=dict(values=initial_temp_df.columns),
    cells=dict(values=[initial_temp_df[col] for col in initial_temp_df.columns])
)])

# Créer une sortie pour le tableau Plotly
table_plot = widgets.Output()
with table_plot:
    initial_fig.show(renderer="notebook")

# Définir une fonction pour mettre à jour le tableau Plotly
def update_plot(change):
    # Filtrer le DataFrame en fonction des valeurs sélectionnées dans les champs de saisie
    temp_df = df.copy()
    filter_list = [temp_df['Sujets'] == sujets.value]
    temp_df = temp_df[reduce(lambda x, y: x & y, filter_list)]
    
    # Mettre à jour le tableau Plotly avec les données filtrées
    with table_plot:
        table_plot.clear_output(wait=True)
        fig = go.Figure(data=[go.Table(
            header=dict(values=temp_df.columns),
            cells=dict(values=[temp_df[col] for col in temp_df.columns])
        )])
        fig.show(renderer="notebook")

# Observer les changements dans les champs de saisie
sujets.observe(update_plot, names="value")

# Ajouter un bouton Reset pour réinitialiser l'affichage à tous les sujets
reset_button = widgets.Button(description="Reset")

def reset_plot(_):
    with table_plot:
        table_plot.clear_output(wait=True)
        initial_fig.show(renderer="notebook")

reset_button.on_click(reset_plot)

# Afficher les champs de saisie, le bouton Reset et le tableau Plotly
widgets.VBox([sujets, reset_button, table_plot])


Atlas

In [245]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets

df = dataframe
# Supposons que vous ayez déjà votre DataFrame df

# Créer un tableau Plotly initial avec toutes les colonnes affichées
initial_fig = go.Figure(data=[go.Table(
    header=dict(values=df.columns),
    cells=dict(values=[df[col] for col in df.columns])
)])

# Créer une sortie pour le tableau Plotly
table_plot = widgets.Output()
with table_plot:
    initial_fig.show(renderer="notebook")

# Définir une fonction pour mettre à jour le tableau Plotly en fonction de la colonne sélectionnée
def update_plot(column_name):
    # Créer un nouveau tableau Plotly avec la colonne sélectionnée uniquement
    fig = go.Figure(data=[go.Table(
        header=dict(values=['Sujets', column_name]),
        cells=dict(values=[df['Sujets'], df[column_name]])
    )])
    
    # Afficher le tableau Plotly dans la sortie
    with table_plot:
        table_plot.clear_output(wait=True)
        fig.show(renderer="notebook")

# Créer des boutons pour chaque colonne (à l'exception de la première) pour afficher sélectivement chaque colonne
buttons = []
for col in df.columns[1:]:
    button = widgets.Button(description=col)
    button.on_click(lambda _, col=col: update_plot(col))
    buttons.append(button)

# Afficher les boutons et le tableau Plotly initial
widgets.VBox([widgets.HBox(buttons), table_plot])


In [ ]:
def get_resources_freesurfer(my_pyxnat, project, subject, experiment):
    my_experiment = my_xnat.select.projects(project).subjects(subject).experiments(experiment)
    for res in my_experiment.resources():
        if res.label() == 'FREESURFER':
            aseg = res.aseg()
            aseg = res.aparc()
            break
